# Projet SDP (Mention IA) — Question 1  
## Explication de type (1-1) avec **Gurobi**

Ce notebook traite **uniquement la Question 1** du sujet : formuler et implémenter un programme d’optimisation linéaire (en nombres entiers) qui **calcule une explication de type (1-1)** pour une comparaison $x \succ y$ si elle existe, et qui **retourne un certificat de non‑existence** sinon.

> Remarque : le solveur utilisé est **Gurobi** via `gurobipy`.


## Partie du sujet — 1. Prologue
Nous nous plaçons dans l’exemple de comparaison entre deux candidats (ici notés `x` et `y`) évalués sur des cours $A\dots G$ avec des poids, et classés par **somme pondérée**.

## Partie du sujet — Données (x et y)  

On encode les notes suivantes (sur 100) et les poids :

- Critères : $A,B,C,D,E,F,G$
- Candidat $x$ : 85, 81, 71, 69, 75, 81, 88  
- Candidat $y$ : 81, 81, 75, 63, 67, 88, 95  
- Poids : 8, 7, 7, 6, 6, 5, 6

On définit la **contribution** de chaque critère $k$ à l’affirmation $x \succ y$ par :
$$
d_k = w_k \,(x_k - y_k)
$$
- $d_k>0$ : critère **pour** $x\succ y$ (ensemble `pros`)  
- $d_k<0$ : critère **contre** $x\succ y$ (ensemble `cons`)  
- $d_k=0$ : critère **neutre**


In [2]:
# Données de l'exemple (x ≻ y)
criteres = ["A","B","C","D","E","F","G"]

notes_x = {"A":85,"B":81,"C":71,"D":69,"E":75,"F":81,"G":88}
notes_y = {"A":81,"B":81,"C":75,"D":63,"E":67,"F":88,"G":95}

poids = {"A":8,"B":7,"C":7,"D":6,"E":6,"F":5,"G":6}

# Contributions d_k = w_k (x_k - y_k)
d = {k: poids[k]*(notes_x[k]-notes_y[k]) for k in criteres}
d


{'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}

## Partie du sujet — 2. Formulation : ensembles pros/cons/neutres
On calcule automatiquement les ensembles `pros(x,y)`, `cons(x,y)` et `neutral(x,y)` à partir des contributions $d_k$.

In [3]:
pros = [k for k in criteres if d[k] > 0]
cons = [k for k in criteres if d[k] < 0]
neutral = [k for k in criteres if d[k] == 0]

print("Contributions d_k :")
for k in criteres:
    print(f"  {k} : {d[k]:+d}")

print("\npros(x,y)   =", pros)
print("cons(x,y)   =", cons)
print("neutral(x,y)=", neutral)


Contributions d_k :
  A : +32
  B : +0
  C : -28
  D : +36
  E : +48
  F : -35
  G : -42

pros(x,y)   = ['A', 'D', 'E']
cons(x,y)   = ['C', 'F', 'G']
neutral(x,y)= ['B']


## Partie du sujet — Trade-offs (1-1)

Un trade-off de type **(1-1)** est une paire $(P,C)$ avec :
- $P \in \mathrm{pros}(x,y)$
- $C \in \mathrm{cons}(x,y)$
- et $d_P + d_C > 0$

On peut donc lister toutes les paires admissibles $(P,C)$ par simple énumération.


In [4]:
tradeoffs_11 = []
for p in pros:
    for c in cons:
        if d[p] + d[c] > 0:
            tradeoffs_11.append((p,c,d[p]+d[c]))

print("Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :")
for (p,c,val) in sorted(tradeoffs_11, key=lambda t: (-t[2], t[0], t[1])):
    print(f"  ({p},{c}) : d_{p}+d_{c} = {val:+d}")

Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :
  (E,C) : d_E+d_C = +20
  (E,F) : d_E+d_F = +13
  (D,C) : d_D+d_C = +8
  (E,G) : d_E+d_G = +6
  (A,C) : d_A+d_C = +4
  (D,F) : d_D+d_F = +1


## Partie du sujet — Question 1 : formulation par optimisation (PLNE)

### Variables de décision
Pour chaque paire admissible $(p,c)$, on introduit une variable binaire :
$$
x_{p,c} \in \{0,1\}
$$
avec $x_{p,c}=1$ si on retient le trade-off $(p,c)$.

### Contraintes
1) **Chaque critère contre** doit être compensé exactement une fois :
$$
\forall c \in \mathrm{cons}(x,y),\quad \sum_{p \in \mathrm{pros}(x,y)} x_{p,c} = 1
$$

2) **Disjonction** : un même critère pour ne peut servir qu’une fois :
$$
\forall p \in \mathrm{pros}(x,y),\quad \sum_{c \in \mathrm{cons}(x,y)} x_{p,c} \le 1
$$

3) Variables définies **uniquement** pour les paires admissibles $(p,c)$ où $d_p + d_c > 0$.

### Objectif
On peut minimiser le nombre de paires sélectionnées :
$$
\min \sum_{(p,c)} x_{p,c}
$$
(avec les contraintes ci-dessus, la solution—si elle existe—utilise en pratique $|\mathrm{cons}(x,y)|$ paires, mais garder un objectif rend la formulation standard.)

### Certificat de non-existence
Si le modèle est **infaisable**, on demande à Gurobi un **IIS** (*Irreducible Inconsistent Subsystem*) : un sous-ensemble minimal de contraintes incompatible. C’est un **certificat de non‑existence** exploitable dans un rendu.


In [5]:
# Implémentation avec Gurobi
# - Si le modèle est faisable : on extrait une explication (1-1)
# - Sinon : on calcule un IIS comme certificat de non-existence

try:
    import gurobipy as gp
    from gurobipy import GRB
except Exception as e:
    raise ImportError(
        "Impossible d'importer gurobipy.\n"
        "Vérifiez que Gurobi est installé et qu'une licence est disponible.\n"
        f"Détail: {e}"
    )

# Construction de l'ensemble des paires admissibles
A = [(p,c) for p in pros for c in cons if d[p] + d[c] > 0]

m = gp.Model("explication_1_1")

# Variables binaires x[p,c] pour (p,c) admissible
xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

# Chaque 'contre' doit être couvert exactement une fois
c_cover = {}
for c in cons:
    c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                             name=f"couvrir_cons[{c}]")

# Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
p_use = {}
for p in pros:
    p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                           name=f"utiliser_pros_au_plus_une_fois[{p}]")

# Objectif : minimiser le nombre de paires sélectionnées
m.setObjective(gp.quicksum(xvar[p,c] for (p,c) in A), GRB.MINIMIZE)

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("\n Une explication (1-1) existe. Paires sélectionnées :\n")
    solution = [(p,c) for (p,c) in A if xvar[p,c].X > 0.5]
    for (p,c) in solution:
        print(f"  - ({p},{c})  avec d_{p}={d[p]:+d} et d_{c}={d[c]:+d}  => somme={d[p]+d[c]:+d}")
    print(f"\nLongueur ℓ = {len(solution)}")

elif m.Status == GRB.INFEASIBLE:
    print("\n Aucune explication (1-1) n'existe : modèle infaisable.")
    print("Calcul d'un IIS (certificat de non-existence) ...")
    m.computeIIS()
    print("\nContraintes appartenant à l'IIS :")
    for constr in m.getConstrs():
        if constr.IISConstr:
            print(" -", constr.ConstrName)
else:
    print("\nStatut inattendu :", m.Status)


Set parameter Username
Set parameter LicenseID to value 2618982
Academic license - for non-commercial use only - expires 2026-02-06
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x0c593c2c
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, be

# Projet SDP — Question 2 : explications de type (1-m)

## Partie du sujet (définition)
On étend le langage d’explication : dans l’affirmation $x \succ y$, un **trade-off de type (1-m)** est une paire $(P,\{C_1,\dots,C_m\})$ où :
- $P \in \mathrm{pros}(x,y)$,
- $C_1,\dots,C_m \in \mathrm{cons}(x,y)$,
- et la somme des contributions est **positive** :
$$
\mathrm{contrib}(P) + \sum_{i=1}^{m}\mathrm{contrib}(C_i) > 0.
$$

Une **explication (1-m)** est un ensemble de trade-offs disjoints
$$
E=\{(P_1,C_1),\dots,(P_\ell,C_\ell)\}
$$
tel que :
- les $P_i$ sont distincts,
- les ensembles $C_i$ sont disjoints,
- et $\bigcup_{i=1}^{\ell} C_i = \mathrm{cons}(x,y)$ (tous les "cons" sont couverts).

## Objectif de la Question 2
Formuler et implémenter un programme d’optimisation qui :
1) calcule une explication (1-m) de $x \succ y$ si elle existe ;
2) retourne un certificat de non-existence sinon (infeasibilité + IIS via le solveur).


In [1]:
# Imports (environnement standard)
import pandas as pd
from collections import defaultdict

# Gurobi
import gurobipy as gp
from gurobipy import GRB


## Partie du sujet (données)
On utilise la table des notes et les poids (A..G) fournis dans l’énoncé, ainsi que l’ordre de classement.
Ici on code ces données en dur pour pouvoir reproduire les comparaisons $x \succ y$, $w \succ w'$, etc.


In [2]:
# Données : notes des candidats (A..G) et poids (weight)
cours = ["A", "B", "C", "D", "E", "F", "G"]

notes = pd.DataFrame(
    {
        "x":  [85, 81, 71, 69, 75, 81, 88],
        "y":  [81, 81, 75, 63, 67, 88, 95],
        "z":  [74, 89, 74, 81, 68, 84, 79],
        "t":  [74, 71, 84, 91, 77, 76, 73],
        "u":  [72, 75, 66, 85, 88, 66, 93],
        "v":  [71, 73, 63, 92, 76, 79, 93],
        "w":  [79, 69, 78, 76, 67, 84, 79],
        "w'": [57, 76, 81, 76, 82, 86, 77],
    },
    index=cours
)

poids = pd.Series([8, 7, 7, 6, 6, 5, 6], index=cours, name="weight")

notes, poids


(    x   y   z   t   u   v   w  w'
 A  85  81  74  74  72  71  79  57
 B  81  81  89  71  75  73  69  76
 C  71  75  74  84  66  63  78  81
 D  69  63  81  91  85  92  76  76
 E  75  67  68  77  88  76  67  82
 F  81  88  84  76  66  79  84  86
 G  88  95  79  73  93  93  79  77,
 A    8
 B    7
 C    7
 D    6
 E    6
 F    5
 G    6
 Name: weight, dtype: int64)

## Contributions, pros/cons/neutral

Pour une comparaison $x \succ y$, la contribution du cours $j$ est :
$$
\mathrm{contrib}(j) = w_j\,(x_j - y_j).
$$

- $\mathrm{pros}(x,y) = \{j : \mathrm{contrib}(j) > 0\}$
- $\mathrm{cons}(x,y) = \{j : \mathrm{contrib}(j) < 0\}$
- $\mathrm{neutral}(x,y) = \{j : \mathrm{contrib}(j) = 0\}$


In [3]:
def contributions(notes: pd.DataFrame, poids: pd.Series, x: str, y: str) -> pd.Series:
    """Calcule contrib(j)=w_j*(x_j - y_j) pour tous les cours j."""
    diff = notes[x] - notes[y]
    return (poids * diff).rename("contribution")

def pros_cons_neutral(contrib: pd.Series):
    pros = [j for j,v in contrib.items() if v > 0]
    cons = [j for j,v in contrib.items() if v < 0]
    neutral = [j for j,v in contrib.items() if v == 0]
    return pros, cons, neutral


## Formulation MILP pour une explication (1-m)

Idée : chaque **pro** $P$ peut "absorber" plusieurs **cons** (un ensemble $C_P$), tant que :
$$
\mathrm{contrib}(P) + \sum_{c\in C_P}\mathrm{contrib}(c) > 0.
$$

### Variables
- $a_{P,c}\in\{0,1\}$ : vaut 1 si le "con" $c$ est affecté au "pro" $P$.
- $s_P\in\{0,1\}$ : vaut 1 si on utilise le "pro" $P$ dans l’explication (donc un trade-off dont le côté négatif est l’ensemble des $c$ affectés à $P$).

### Contraintes
1) **Couverture des cons (partition)** : chaque $c\in \mathrm{cons}(x,y)$ doit être expliqué exactement une fois
$$
\sum_{P\in \mathrm{pros}(x,y)} a_{P,c} = 1 \quad \forall c\in \mathrm{cons}(x,y).
$$

2) **Lien** : si un con est affecté à $P$, alors $P$ est utilisé
$$
a_{P,c} \le s_P \quad \forall P,\forall c.
$$

3) **Positivité du trade-off (strictement)** : si $P$ est utilisé, alors la somme des contributions du bloc $(P, C_P)$ est > 0.
Comme ici les contributions sont entières (poids et notes entiers), on remplace $>0$ par $\ge 1$ :
$$
\mathrm{contrib}(P)\,s_P + \sum_{c\in \mathrm{cons}} \mathrm{contrib}(c)\,a_{P,c} \ge 1\cdot s_P \quad \forall P.
$$

### Objectif
Minimiser le nombre de trade-offs (la longueur de l’explication) :
$$
\min \sum_{P\in \mathrm{pros}} s_P.
$$

Si le modèle est **infeasible**, on calcule un **IIS** (Irreducible Inconsistent Subsystem) comme certificat pratique.


In [4]:
def expliquer_1m_gurobi(notes: pd.DataFrame, poids: pd.Series, x: str, y: str, verbose: bool = False):
    """
    Retourne une explication (1-m) pour x ≻ y si elle existe.
    Sinon, calcule un IIS comme certificat d'infeasibilité.
    """
    contrib = contributions(notes, poids, x, y)
    pros, cons, neutral = pros_cons_neutral(contrib)

    if len(cons) == 0:
        return {
            "statut": "trivial",
            "message": "Aucun 'con' : l'explication est vide (x domine déjà sans désavantage pondéré).",
            "contrib": contrib, "pros": pros, "cons": cons, "neutral": neutral, "explication": []
        }

    if len(pros) == 0:
        return {
            "statut": "impossible",
            "message": "Aucun 'pro' mais des 'cons' existent : impossible de construire une explication (1-m).",
            "contrib": contrib, "pros": pros, "cons": cons, "neutral": neutral
        }

    m = gp.Model("explication_1m")
    m.Params.OutputFlag = 1 if verbose else 0

    # Variables
    s = {P: m.addVar(vtype=GRB.BINARY, name=f"s[{P}]") for P in pros}
    a = {(P,c): m.addVar(vtype=GRB.BINARY, name=f"a[{P},{c}]") for P in pros for c in cons}

    # (1) Couverture exacte des cons
    for c in cons:
        m.addConstr(gp.quicksum(a[P,c] for P in pros) == 1, name=f"couvrir[{c}]")

    # (2) Lien a[P,c] <= s[P]
    for P in pros:
        for c in cons:
            m.addConstr(a[P,c] <= s[P], name=f"lien[{P},{c}]")

    # (3) Positivité stricte (contributions entières => >= 1 si s[P]=1)
    for P in pros:
        m.addConstr(
            contrib[P]*s[P] + gp.quicksum(contrib[c]*a[P,c] for c in cons) >= 1*s[P],
            name=f"positif[{P}]"
        )

    # Objectif : minimiser le nombre de pros utilisés (= longueur de l'explication)
    m.setObjective(gp.quicksum(s[P] for P in pros), GRB.MINIMIZE)

    m.optimize()

    if m.Status == GRB.OPTIMAL:
        explication = []
        for P in pros:
            cons_affectes = [c for c in cons if a[P,c].X > 0.5]
            if cons_affectes:
                # Vérification du trade-off
                somme = contrib[P] + sum(contrib[c] for c in cons_affectes)
                explication.append((P, cons_affectes, somme))
        explication.sort(key=lambda t: t[0])

        return {
            "statut": "ok",
            "contrib": contrib, "pros": pros, "cons": cons, "neutral": neutral,
            "objectif": m.ObjVal,
            "explication": explication
        }

    # Infeasible => IIS
    if m.Status == GRB.INFEASIBLE:
        m.computeIIS()
        contraintes_iis = []
        for constr in m.getConstrs():
            if constr.IISConstr:
                contraintes_iis.append(constr.ConstrName)

        return {
            "statut": "infeasible",
            "contrib": contrib, "pros": pros, "cons": cons, "neutral": neutral,
            "iis": contraintes_iis
        }

    return {
        "statut": "autre",
        "message": f"Statut Gurobi inattendu: {m.Status}",
        "contrib": contrib, "pros": pros, "cons": cons, "neutral": neutral
    }


## Exécution : exemple sur la comparaison $w \succ w'$

Le sujet mentionne qu’un trade-off (1-3) de $w \succ w'$ est $(A,\{B,C,E\})$.
On teste si une **explication complète (1-m)** existe, et on extrait les blocs $(P, C_P)$ trouvés.


In [5]:
res = expliquer_1m_gurobi(notes, poids, "w", "w'", verbose=False)

print("Statut :", res["statut"])
print("\nContributions :")
display(res["contrib"].to_frame())

print("\npros =", res["pros"])
print("cons =", res["cons"])
print("neutral =", res["neutral"])

if res["statut"] == "ok":
    print("\nExplication (1-m) trouvée (format: (P, {cons}, somme_contributions)) :")
    for P, Cs, somme in res["explication"]:
        print(f"  ({P}, {{{', '.join(Cs)}}})  somme = {somme}")
    print("\nLongueur (nombre de trade-offs) =", int(res["objectif"]))
elif res["statut"] == "infeasible":
    print("\nAucune explication (1-m). IIS (liste de contraintes impliquées) :")
    for name in res["iis"]:
        print("  -", name)
else:
    print(res.get("message", ""))


Set parameter Username
Set parameter LicenseID to value 2618982
Academic license - for non-commercial use only - expires 2026-02-06
Statut : ok

Contributions :


,contribution
A,176
B,-49
C,-21
D,0
E,-90
F,-10
G,12



pros = ['A', 'G']
cons = ['B', 'C', 'E', 'F']
neutral = ['D']

Explication (1-m) trouvée (format: (P, {cons}, somme_contributions)) :
  (A, {B, C, E, F})  somme = 6

Longueur (nombre de trade-offs) = 1


## Exécution : autre comparaison (par exemple $x \succ y$)

Vous pouvez changer les deux candidats ci-dessous pour tester d’autres comparaisons du classement.


In [14]:
res_xy = expliquer_1m_gurobi(notes, poids, "x", "y", verbose=False)

print("Statut :", res_xy["statut"])
display(res_xy["contrib"].to_frame())
print("pros =", res_xy["pros"])
print("cons =", res_xy["cons"])

if res_xy["statut"] == "ok":
    print("\nExplication (1-m) :")
    for P, Cs, somme in res_xy["explication"]:
        print(f"  ({P}, {{{', '.join(Cs)}}})  somme = {somme}")
    print("Longueur =", int(res_xy["objectif"]))
elif res_xy["statut"] == "infeasible":
    print("\nInfeasible. IIS :")
    for name in res_xy["iis"]:
        print("  -", name)


Statut : ok


,contribution
A,32
B,0
C,-28
D,36
E,48
F,-35
G,-42


pros = ['A', 'D', 'E']
cons = ['C', 'F', 'G']

Explication (1-m) :
  (A, {C})  somme = 4
  (D, {F})  somme = 1
  (E, {G})  somme = 6
Longueur = 3
